In [105]:
import numpy as np
import pandas as pd
import re # regular expression: searching for words in a paragraph or text
from nltk.corpus import stopwords #  natural language toolkit. Stopwords: words that are not important (ex: a, and, where, what,...)
from nltk.stem.porter import PorterStemmer # stemming: removes prefic and suffix from word, then returns its root
from sklearn.feature_extraction.text import TfidfVectorizer #  convert text to feature vectors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [106]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [107]:
# words with no value
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

#**Data Preprocessing**

In [108]:
data = pd.read_csv('/content/train.csv')

In [109]:
data.shape

(20800, 5)

In [110]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


- Features:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete

- labels:
   
   1: Fake news
   
   0: real News





In [111]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [112]:
# remove missing values as they are not really important (replacing null values with empty string)
data = data.fillna('')

In [113]:
data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [114]:
data.duplicated().sum()

0

In [115]:
# merge author name and titles to use them for prediction as texts are enormous
data['content'] = data['author'] + ' ' + data['title']

In [116]:
data['content']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

#**Stemming**
the process of returning a verb to its original verb

ex: actor, acting, actress --> act

In [117]:
stemp = PorterStemmer()

In [118]:
def stemming(content):
  st = re.sub('[^a-zA-z]', ' ', content) # ^: removing every thing from 'content' that is outside of the bound like numbers, snd replace then with ' '
  st = st.lower()
  st = st.split()
  st = [stemp.stem(word) for word in st if not word in stopwords.words('english')]  # portstemming all the words except stepwords
  st = ' '.join(st)
  return st

In [119]:
data['content'] = data['content'].apply(stemming)

In [120]:
data['content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [121]:
x

<20800x17141 sparse matrix of type '<class 'numpy.float64'>'
	with 210632 stored elements in Compressed Sparse Row format>

In [122]:
# seperate data and label
x = data['content'].values
y = data['label'].values

In [123]:
x

array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exercis balkan',
       'david swanson keep f aliv'], dtype=object)

In [124]:
y

array([1, 0, 1, ..., 0, 1, 1])

In [125]:
x.shape

(20800,)

In [126]:
x.shape

(20800,)

#**Vectorization**

In [127]:
# convirt text to numbers for computer to understand
vec = TfidfVectorizer() # count the repetition of certain words aka it is an important word, then give it a certian value

In [128]:
vec.fit(x,y)

TfidfVectorizer()

In [129]:
x = vec.transform(x)

In [130]:
print(x)

  (0, 15697)	0.28485063562728646
  (0, 13480)	0.2565896679337957
  (0, 8915)	0.3635963806326075
  (0, 8636)	0.29212514087043684
  (0, 7698)	0.24785219520671603
  (0, 7012)	0.21874169089359144
  (0, 4979)	0.233316966909351
  (0, 3795)	0.2705332480845492
  (0, 3603)	0.3598939188262559
  (0, 2962)	0.2468450128533713
  (0, 2485)	0.3676519686797209
  (0, 268)	0.27010124977708766
  (1, 16812)	0.30071745655510157
  (1, 6823)	0.1904660198296849
  (1, 5510)	0.7143299355715573
  (1, 3571)	0.26373768806048464
  (1, 2816)	0.19094574062359204
  (1, 2224)	0.3827320386859759
  (1, 1895)	0.15521974226349364
  (1, 1498)	0.2939891562094648
  (2, 15622)	0.41544962664721613
  (2, 9625)	0.49351492943649944
  (2, 5975)	0.3474613386728292
  (2, 5396)	0.3866530551182615
  (2, 3106)	0.46097489583229645
  :	:
  (20797, 13128)	0.2482526352197606
  (20797, 12350)	0.27263457663336677
  (20797, 12144)	0.24778257724396507
  (20797, 10311)	0.08038079000566466
  (20797, 9593)	0.174553480255222
  (20797, 9523)	0.295420

#**Training the Model**

In [131]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.2, stratify=y, random_state=2)

In [132]:
model = LogisticRegression()

In [133]:
model.fit(X_train,Y_train)

LogisticRegression()

#**Evaluating the Model**

In [134]:
xpred = model.predict(X_train)
acc = accuracy_score(xpred, Y_train)
acc

0.9864783653846154

In [135]:
xpredd = model.predict(X_test)
acc2 = accuracy_score(xpredd, Y_test)
acc2

0.9790865384615385

#**Predictive System**

In [149]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [150]:
print(Y_test[3])

0
